In [125]:
import os
import pandas as pd
import glob
import numpy as np
import datetime
import seaborn as sns
from datetime import datetime, timedelta

import warnings; 
warnings.filterwarnings("ignore")

In [129]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from urllib.parse import quote
import datetime
import seaborn as sns
from datetime import datetime, timedelta
import datetime
import matplotlib.pyplot as plt

import statsmodels.api as sm
from pylab import rcParams

import warnings; 
warnings.filterwarnings("ignore")

In [130]:
cur_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("current times:", cur_time)

current times: 2024-07-24 21:13:12


In [82]:
# Specify the directory containing the CSV files
directory = r'data/temp/'  # Use raw string to handle backslashes

amp = [
    'BUKT.csv',
    'CIHO.csv',
    'SAY001.csv',
    'SAY002.csv',
    'STH005.csv',
    'STH007.csv',
    'STH010.csv',
    'STH011.csv',
    'STH013.csv',
    'STH014.csv',
    'STH019.csv',
    'STH021.csv',
    'STH022.csv',
    'STH023.csv',
    'STH025.csv',
    'STH026.csv'
]
# Read each CSV file into a DataFrame, add a new column, and store them in a list
dataframes = []
for dirpath, _, filenames in os.walk(directory):
    for file in filenames:
        if file.endswith('.csv'):
            if file in amp:
                # print(file)
                file_path = os.path.join(dirpath, file)
                try:
                    df = pd.read_csv(file_path)
                    # df['source_file'] = os.path.relpath(file_path, directory)  # Add a new column with the relative file path
                    df['source_file'] = file
                    dataframes.append(df)
                except FileNotFoundError:
                    print(f"File not found: {file_path}")
                except pd.errors.EmptyDataError:
                    print(f"File is empty: {file_path}")
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")

# Optionally, concatenate all DataFrames into a single DataFrame
if dataframes:
    all_data = pd.concat(dataframes, ignore_index=True)
    # Now 'all_data' contains all the data from the CSV files with an additional column 'source_file'
    print(all_data)
else:
    print("No dataframes to concatenate.")

              date   time  temp source_file
0       2018-01-01  00:00  25.7    BUKT.csv
1       2018-01-01  01:00  25.7    BUKT.csv
2       2018-01-01  02:00  25.5    BUKT.csv
3       2018-01-01  03:00  25.1    BUKT.csv
4       2018-01-01  04:00  25.1    BUKT.csv
...            ...    ...   ...         ...
841339  2023-12-31  19:00  25.2  STH026.csv
841340  2023-12-31  20:00    24  STH026.csv
841341  2023-12-31  21:00  23.3  STH026.csv
841342  2023-12-31  22:00  22.9  STH026.csv
841343  2023-12-31  23:00  22.7  STH026.csv

[841344 rows x 4 columns]


In [83]:
temp = pd.DataFrame(all_data)

In [16]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841344 entries, 0 to 841343
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   date         841344 non-null  object
 1   time         841344 non-null  object
 2   temp         841344 non-null  object
 3   source_file  841344 non-null  object
dtypes: object(4)
memory usage: 25.7+ MB


In [84]:
# from datetime import datetime
def calculate_week_number(str_date):
    # Start of the custom week 1
    # str_date = datetime.strptime(str_date, format).date()
    start_of_week_1 = pd.Timestamp(year=str_date.year, month=1, day=7)
    # print(start_of_week_1)
    # if str_date < start_of_week_1:
    #     start_date = datetime.date(str_date.year, 1, 7)
    if str_date < start_of_week_1:
        return 52  # For dates before the start of week 1
    return ((str_date - start_of_week_1).days // 7) + 1

In [86]:
# Strip any leading/trailing whitespace from the date strings
temp['date'] = temp['date'].str.strip()
# temp['date'] = datetime.strptime(temp['date'], '%Y-%m-%d')
# Convert 'DATESICK' column to datetime
temp['date'] = pd.to_datetime(temp['date'], format='%Y-%m-%d')

In [87]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841344 entries, 0 to 841343
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         841344 non-null  datetime64[ns]
 1   time         841344 non-null  object        
 2   temp         841344 non-null  object        
 3   source_file  841344 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 25.7+ MB


In [88]:
# Apply the function to get the week number
temp['week_number'] = temp['date'].apply(calculate_week_number)

In [89]:
temp.tail()

,date,time,temp,source_file,week_number
841339,2023-12-31,19:00,25.2,STH026.csv,52
841340,2023-12-31,20:00,24,STH026.csv,52
841341,2023-12-31,21:00,23.3,STH026.csv,52
841342,2023-12-31,22:00,22.9,STH026.csv,52
841343,2023-12-31,23:00,22.7,STH026.csv,52


In [93]:
# Replace -999.0 with NaN
temp['temp'] = temp['temp'].replace(-999.0, np.nan)

In [98]:
# Convert 'temp' column to numeric, coercing errors to NaN
temp['temp'] = pd.to_numeric(temp['temp'], errors='coerce')

In [99]:
t = temp[temp['source_file'] == 'STH019.csv']
t.head(24)

,date,time,temp,source_file,week_number
7440,2018-01-01,00:00,34.8,STH019.csv,52
7441,2018-01-01,01:00,33.8,STH019.csv,52
7442,2018-01-01,02:00,32.8,STH019.csv,52
7443,2018-01-01,03:00,32.3,STH019.csv,52
7444,2018-01-01,04:00,30.4,STH019.csv,52
7445,2018-01-01,05:00,30.9,STH019.csv,52
7446,2018-01-01,06:00,31.4,STH019.csv,52
7447,2018-01-01,07:00,32.3,STH019.csv,52
7448,2018-01-01,08:00,NaN,STH019.csv,52
7449,2018-01-01,09:00,NaN,STH019.csv,52


หาค่าเฉลี่ยของวัน

In [105]:
temp['temp'] = pd.to_numeric(temp['temp'])
temp_mean = temp.groupby(['date','source_file','week_number'])["temp"].mean().reset_index(name='temp_mean')
temp_mean.head(30)

,date,source_file,week_number,temp_mean
0,2018-01-01,BUKT.csv,52,25.837500
1,2018-01-01,CIHO.csv,52,25.670833
2,2018-01-01,SAY001.csv,52,NaN
3,2018-01-01,SAY002.csv,52,NaN
4,2018-01-01,STH005.csv,52,NaN
5,2018-01-01,STH007.csv,52,26.591667
6,2018-01-01,STH010.csv,52,NaN
7,2018-01-01,STH011.csv,52,27.000000
8,2018-01-01,STH013.csv,52,27.216667
9,2018-01-01,STH014.csv,52,24.456522


หาค่าเฉลี่ยของ week รายอำเภอ

In [ ]:
temp_mean_week = temp.groupby(['source_file','week_number'])["temp"].mean().reset_index(name='temp_mean')
temp_mean_week.tail(30)

หาค่าเฉลี่ยของ week ทั้งจังหวัด

In [136]:
temp_mean_week = temp.groupby(['date'])["temp"].mean().reset_index(name='temp_mean')
temp_mean_week.head(10)

,date,temp_mean
0,2018-01-01,26.359048
1,2018-01-02,26.446635
2,2018-01-03,26.889100
3,2018-01-04,26.902392
4,2018-01-05,28.096135
5,2018-01-06,27.980788
6,2018-01-07,28.097608
7,2018-01-08,27.349545
8,2018-01-09,27.638835
9,2018-01-10,26.997608


In [137]:
# Apply the function to get the week number
temp_mean_week['week_number'] = temp_mean_week['date'].apply(calculate_week_number)

In [138]:
temp_mean_week.head()

,date,temp_mean,week_number
0,2018-01-01,26.359048,52
1,2018-01-02,26.446635,52
2,2018-01-03,26.889100,52
3,2018-01-04,26.902392,52
4,2018-01-05,28.096135,52


In [140]:
temp_mean_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         2191 non-null   datetime64[ns]
 1   temp_mean    2191 non-null   float64       
 2   week_number  2191 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 51.5 KB


In [142]:
temp_mean_week['YEAR'] = temp_mean_week['date'].dt.year
temp_mean_week.head()

,date,temp_mean,week_number,YEAR
0,2018-01-01,26.359048,52,2018
1,2018-01-02,26.446635,52,2018
2,2018-01-03,26.889100,52,2018
3,2018-01-04,26.902392,52,2018
4,2018-01-05,28.096135,52,2018


In [146]:
# Group by week number and sum cases
#weekly_cases = df.groupby('week_number','NADDRCODE')['DATESICK'].count().reset_index()
temp_on_week_in_year = temp_mean_week.groupby(['week_number','YEAR'])["temp_mean"].mean().reset_index(name='temp_mean')
print(temp_on_week_in_year)

     week_number  YEAR  temp_mean
0              1  2018  26.329732
1              1  2019  27.282259
2              1  2020  26.972277
3              1  2021  24.893962
4              1  2022  26.614588
..           ...   ...        ...
307           52  2019  26.933123
308           52  2020  26.442933
309           52  2021  24.746637
310           52  2022  26.318204
311           52  2023  26.361806

[312 rows x 3 columns]


In [147]:
def find_week_number(date):
    """
    Calculate the week number of a given date, where week 1 starts on January 7.

    Args:
    date (datetime.date): The date to calculate the week number for.

    Returns:
    int: The week number of the date.
    """
    # Define the starting date of week 1
    start_date = datetime.date(date.year, 1, 7)
    
    # If the given date is before January 7 of the same year, adjust the start date to the previous year's January 7
    if date < start_date:
        start_date = datetime.date(date.year - 1, 1, 7)
    
    # Calculate the difference in days
    days_diff = (date - start_date).days
    
    # Calculate the week number
    week_number = (days_diff // 7) + 1
    
    return week_number

def get_start_of_week(week_number, year):
    """
    Calculate the start date of a given week number, where week 1 starts on January 7.

    Args:
    week_number (int): The week number.
    year (int): The year.

    Returns:
    datetime.date: The start date of the week.
    """
    start_date = datetime.date(year, 1, 7)
    start_of_week = start_date + datetime.timedelta(weeks=week_number - 1)
    return start_of_week

def create_weeks_dataframe(start_date_str, end_date_str):
    """
    Create a DataFrame with weeks starting from January 7, along with their start and end dates.

    Args:
    start_date_str (str): The start date string for the DataFrame in 'YYYY-MM-DD' format.
    end_date_str (str): The end date string for the DataFrame in 'YYYY-MM-DD' format.

    Returns:
    pd.DataFrame: A DataFrame with week numbers and their start and end dates.
    """
    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d').date()
    
    weeks = []
    current_date = start_date
    
    while current_date <= end_date:
        week_number = find_week_number(current_date)
        start_of_week = get_start_of_week(week_number, current_date.year)
        end_of_week = start_of_week + datetime.timedelta(days=6)
        weeks.append({
            'week_number': week_number,
            'start_date': start_of_week,
            'end_date': end_of_week
        })
        current_date = end_of_week + datetime.timedelta(days=1)
    
    weeks_df = pd.DataFrame(weeks).drop_duplicates(subset=['week_number'])
    return weeks_df

In [150]:
# Example usage:
start_date = '2024-01-07'
end_date = '2025-01-04'
weeks_df = create_weeks_dataframe(start_date, end_date)
weeks_df

,week_number,start_date,end_date
0,1,2024-01-07,2024-01-13
1,2,2024-01-14,2024-01-20
2,3,2024-01-21,2024-01-27
3,4,2024-01-28,2024-02-03
4,5,2024-02-04,2024-02-10
5,6,2024-02-11,2024-02-17
6,7,2024-02-18,2024-02-24
7,8,2024-02-25,2024-03-02
8,9,2024-03-03,2024-03-09
9,10,2024-03-10,2024-03-16


In [149]:
temp_on_week_in_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   week_number  312 non-null    int64  
 1   YEAR         312 non-null    int32  
 2   temp_mean    312 non-null    float64
dtypes: float64(1), int32(1), int64(1)
memory usage: 6.2 KB


In [152]:
ndf = temp_on_week_in_year.merge(weeks_df, on='week_number', how='left')
ndf.head()

,week_number,YEAR,temp_mean,start_date,end_date
0,1,2018,26.329732,2024-01-07,2024-01-13
1,1,2019,27.282259,2024-01-07,2024-01-13
2,1,2020,26.972277,2024-01-07,2024-01-13
3,1,2021,24.893962,2024-01-07,2024-01-13
4,1,2022,26.614588,2024-01-07,2024-01-13


In [154]:
new_df2 = pd.merge(temp_on_week_in_year, weeks_df , on=['week_number'])
new_df2 = new_df2[['week_number','YEAR', 'start_date', 'end_date', 'temp_mean']]
new_df2.head()

,week_number,YEAR,start_date,end_date,temp_mean
0,1,2018,2024-01-07,2024-01-13,26.329732
1,1,2019,2024-01-07,2024-01-13,27.282259
2,1,2020,2024-01-07,2024-01-13,26.972277
3,1,2021,2024-01-07,2024-01-13,24.893962
4,1,2022,2024-01-07,2024-01-13,26.614588


In [157]:
# Strip any leading/trailing whitespace from the date strings
# new_df2['start_date'] = new_df2['start_date'].str.strip()

# Convert 'DATESICK' column to datetime
new_df2['start_date'] = pd.to_datetime(new_df2['start_date'], format='%d/%m/%Y')
new_df2['MONTH'] = new_df2['start_date'].dt.month
new_df2['DAY'] = new_df2['start_date'].dt.day
new_df2.head()

,week_number,YEAR,start_date,end_date,temp_mean,MONTH,DAY
0,1,2018,2024-01-07,2024-01-13,26.329732,1,7
1,1,2019,2024-01-07,2024-01-13,27.282259,1,7
2,1,2020,2024-01-07,2024-01-13,26.972277,1,7
3,1,2021,2024-01-07,2024-01-13,24.893962,1,7
4,1,2022,2024-01-07,2024-01-13,26.614588,1,7


In [159]:
# new_df2['DATE'] = pd.to_datetime(new_df2[['YEARSICK','MONTHSICK','DAYSICK']])
new_df2['date'] = pd.to_datetime(new_df2.YEAR.astype(str) + '/' + new_df2.MONTH.astype(str) +'/' + new_df2.DAY.astype(str))

In [160]:
df2 = new_df2[['date','temp_mean']].sort_values('date')

In [161]:
df2.rename(columns={'date': "date", 'temp_mean' : "temp"}, inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 312 entries, 0 to 311
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    312 non-null    datetime64[ns]
 1   temp    312 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 7.3 KB


In [163]:
df2.to_csv(r"data\dataset\temp.csv", index=False)